###Spark Aggregate Functions

####Funciones Simples de agregación

In [0]:
%run "../includes/configuration"

In [0]:
movies_df = spark.read.parquet(f"{silver_folder_path}/movies")

In [0]:
country_df = spark.read.parquet(f"{silver_folder_path}/countries")

In [0]:
production_country_df = spark.read.parquet(f"{silver_folder_path}/productions_countries")

In [0]:
movies_filter_df = movies_df.filter("year_release_date >= 2015")

In [0]:
country_prod_coun_df = country_df.join(production_country_df,
                                       country_df.country_id == production_country_df.country_id, "inner") \
                                    .select(country_df.country_name, production_country_df.movie_id)

In [0]:
results_movie_country_prod_coun_df = movies_filter_df.join(country_prod_coun_df, 
                                                           movies_filter_df.movie_id == country_prod_coun_df.movie_id, "inner")

In [0]:
results_df = results_movie_country_prod_coun_df.select("year_release_date", "country_name", "budget", "revenue")

In [0]:
from pyspark.sql.functions import sum

results_group_by_df = results_df.groupBy("year_release_date", "country_name") \
                                  .agg(
                                        sum("budget").alias("total_budget"),
                                        sum("revenue").alias("total_revenue")
                                    )

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, dense_rank

In [0]:
result_dense_rank_df = Window.partitionBy("year_release_date").orderBy(desc("total_budget"), desc("total_revenue"))

final_df =  results_group_by_df.withColumn("dense_rank", dense_rank().over(result_dense_rank_df))


In [0]:
final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_gold.result_group_movie_country")
#final_df.write.mode("overwrite").parquet(f"{gold_folder_path}/result_group_movie_country")

In [0]:
display(spark.read.parquet(f"{gold_folder_path}/result_group_movie_country"))